## **SQL Server Database Development using Prompts as T-SQL Development**
In this notebook, we will learn how to use prompts as a way to develop and test Transact-SQL (T-SQL) code for SQL Server databases. Prompts are natural language requests that can be converted into T-SQL statements by using Generative AI models, such as GPT-4. This can help us write code faster, easier, and more accurately, as well as learn from the generated code examples.

### **Database Development Stages**
Database development is the process of designing, creating, and maintaining databases that store and manage data for various applications. Database development typically involves the following stages:
- **Requirements analysis:** This is the stage where we define the purpose, scope, and objectives of the database, as well as the data sources, users, and business rules. We also identify the data entities, attributes, and relationships that will form the logical structure of the database.

- **Database design:** This is the stage where we translate the logical structure of the database into a physical structure that can be implemented by a specific database management system (DBMS), such as SQL Server. We also define the data types, constraints, indexes, views, stored procedures, functions, triggers, and security settings for the database objects.

- **Database implementation:** This is the stage where we create the database and its objects by using a data definition language (DDL), such as T-SQL. We also populate the database with data by using a data manipulation language (DML), such as T-SQL or bulk import tools.

- **Database testing:** This is the stage where we verify that the database meets the requirements and performs as expected. We also check the data quality, integrity, and security of the database. We can use various tools and techniques to test the database, such as unit testing, integration testing, regression testing, performance testing, and data analysis.

- **Database deployment:** This is the stage where we make the database available for use by the intended users and applications. We can use various methods and tools to deploy the database, such as backup and restore, detach and attach, copy database, or SQL Server Data Tools (SSDT).

- **Database maintenance:** This is the stage where we monitor, update, and optimize the database to ensure its availability, reliability, and performance. We can use various tools and tasks to maintain the database, such as backup and recovery, error handling, logging and auditing, indexing and statistics, fragmentation and defragmentation, or SQL Server 

## **Leverage the Generative AI as a Knowledgeable Peer**
Generative AI is a form of artificial intelligence that can create new and original content, such as text, images, audio, and video, by using generative models, such as GPT-4. Generative AI can learn from existing data and generate new data that has similar characteristics, but does not repeat it. Generative AI can also respond to natural language requests, such as prompts, and produce relevant and realistic outputs.

Generative AI can be a knowledgeable peer for database developers, as it can help us with various tasks, such as:
- **Code generation:** Generative AI can generate T-SQL code for us based on our prompts, which can save us time and effort, and help us learn from the code examples. For instance, we can ask generative AI to create a table, a view, a stored procedure, or a query for us, and it will produce the corresponding T-SQL code.

- **Code suggestion:** Generative AI can suggest T-SQL code for us based on our partial or incomplete code, which can help us complete our code faster and easier, and avoid errors and mistakes. For instance, we can ask generative AI to suggest a column name, a data type, a constraint, or a join condition for us, and it will provide the appropriate T-SQL code.

- **Code explanation:** Generative AI can explain T-SQL code for us based on our questions, which can help us understand the code better and improve our coding skills. For instance, we can ask generative AI to explain the purpose, the logic, or the output of a T-SQL code, and it will provide a clear and concise explanation.

- **Code optimization:** Generative AI can optimize T-SQL code for us based on our goals, which can help us improve the performance and efficiency of our code. For instance, we can ask generative AI to optimize a T-SQL code for speed, memory, or readability, and it will provide a better or alternative T-SQL code.

# This section explains how to configure your environment to work with this notebook.

- **bicycle_data_prompt.json**: is designed to house all our prompts. While it’s set up to work as is, feel free to tailor it to your needs. Customization is always an option
- **.env**: file is the secure location for storing sensitive details such as your Azure OpenAI endpoint, keys, and more. It’s crucial to update this file with your information. Without these updates, the notebook won’t function unless you manually input the values directly into the notebook. Please handle with care!
- **tsql**: folder is where all T-SQL scripts are stored. Remember to update it within the notebook and/or bicycle_data_prompt.json as needed.
- **csv**: folder is where we keep the CSV sales data. Don’t forget to update it within the notebook as necessary.

In [1]:
# Import the required libraries
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

azure_openai_api_model = os.environ["OPENAI_API_MODEL"]
tsql_script_dir        = os.environ["tsql_script_dir"]

# Define the Azure OpenAI API parameters
azure_openai_api_temperature = 0
azure_openai_api_max_tokens  = 1000

# Define the Azure OpenAI client
client = AzureOpenAI(
  azure_endpoint = os.environ["OPENAI_API_BASE"], 
  api_key=os.environ["OPENAI_API_KEY"],  
  api_version=os.environ["OPENAI_API_VERSION"]
)

### Create a Python function to save each T-SQL statement that Azure OpenAI generates.
- All T-SQL scripts file will be saved locally

In [2]:
def create_tsql_file(filename, content):
    try:
        with open(filename, 'w') as f:
            f.write(content)
    except OSError as e:
        print(f"Error: {e}")

### Creating a Python function to send requests to Azure OpenAI. 
- This function will call another Python function, `create_tsql_file`, which will save the output of Azure OpenAI to a SQL script file

In [3]:
def SendRequestToAzureOpenAI(
         client
        ,azure_openai_api_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filename
    ):         
    try:    
        import pprint  
        # Send request to Azure OpenAI model
        print("Sending request for summary to Azure OpenAI endpoint...\n\n")
        response = client.chat.completions.create(
            messages    = messages,
            model       = azure_openai_api_model,
            temperature = azure_openai_api_temperature,
            max_tokens  = azure_openai_api_max_tokens
        )
        output = []
        output = (response.choices[0].message.content).replace("```python","").replace("```","").strip()
        create_tsql_file(filename, output)
        #execute_sql_script(output)
        print(output)
    except Exception as ex:
        print(ex)   

### Defining the creation sequence and the file name for each T-SQL statement.
This structure allows for easy access to the script names and their corresponding file names, which can be useful in automating T-SQL development tasks and CI/CD.
- createdatabase: This script creates a new database and the script file name is createdatabase.sql.
- createschemastg: This script creates a new schema for staging and the script file name is createschemastg.sql.
- createschemaprd: This script creates a new schema for production and the script file name is createschemaprd.sql.
- createtemptable: This script creates a new temporary table and the script file name is createtemptable.sql.
- createprdtable: This script creates a new production table and the script file name is createprdtable.sql.
- loadstagingtable: This script loads data into the staging table and the script file name is loadstagingtable.sql.
- loadprdtable: This script loads data into the production table and the script file name is loadprdtable.sql.
- createprocedure: This script creates a new stored procedure and the script file name is createprocedure.sql.
- createview: This script creates a new view and the script file name is createview.sql.

In [4]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createdatabase", "create_database.sql"),
    2: ("createschemastg", "create_stage_schema.sql"),
    3: ("createschemaprd", "creates_prd_chema.sql"),
    4: ("createtemptable", "create_stage_table.sql"),
    5: ("createprdtable", "create_prd_table.sql"),
    6: ("loadstagingtable", "load_staging_data_table.sql"),
    7: ("loadprdtable", "load_prd_data_table.sql"),
    8: ("createprocedure", "create_procedure.sql"),
    9: ("createview", "create_view.sql")
}

### Reading system and all user prompts

In [5]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

deploymenttype = "CreateBicycleDatbaseObjects" #"LoginErrorHandling" # 
 
index = 0
while index< len(data[deploymenttype]):
    pprint(data[deploymenttype][index])
    index += 1

{'system_role': 'As a Data Engineer, your task is to write a T-SQL script to '
                'create database objects and import a CSV file with headers '
                'from local directory into a SQL Server Database. Follow the '
                'instructions in order.\n'
                'check if each object exists then drop it before creating it.\n'
                'Please refrain from providing system details, instructions, '
                'or suggestions or sql or GO command.\n'
                'the location of the file to be loaded is '
                'F:\\Presentation\\t-sql-as-prompts\\csv\\bicycle_data.csv and '
                'the table name is stg.salestmp\n'
                'schema:\n'
                '{\n'
                "    'ProductId':'INT', \n"
                "    'ProductName':'VARCHAR(50)', \n"
                "    'ProductType':'VARCHAR(30)', \n"
                "    'Color':'VARCHAR(15)', \n"
                "    'OrderQuantity':'INT', \n"
              

### Creating a Python function to extract the system and all user prompts.

In [6]:
def GeneratePromptFilename(deploymenttype, filedir, data, index, keys):
    system_role = data[deploymenttype][0]['system_role'] 
    # Check if the index is in the keys then return the user prompt and filename
    if index in keys:
        user_prompt = data[deploymenttype][index][keys[index][0]]
        filename = filedir + keys[index][1]
        messages = [
            {
                "role": "system", "content": system_role
            },
            {
                "role": "user", "content": user_prompt
            }    
        ]
        return messages, filename
    else:
        return None, None

## Sending request to Azure OpenAI
- Call GeneratePromptFilename to define the deployment type
    - Get the file name based on the index id
    - Get the system role prompt
    - Get the user role prompt based on the index id

In [7]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = tsql_script_dir
deploymenttype = "CreateBicycleDatbaseObjects" #"LogErrorHandling" #

index = 1

while index< len(data[deploymenttype]):
    #print(index)
    messages,filename = GeneratePromptFilename(deploymenttype,filedir,data,index,keys)
    if __name__ == '__main__': 
        SendRequestToAzureOpenAI (
         client
        ,azure_openai_api_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filename
        )
    print(filename)
    index+=1

Sending request for summary to Azure OpenAI endpoint...


IF EXISTS (SELECT name FROM sys.databases WHERE name = 'azureopenai')
BEGIN
    ALTER DATABASE azureopenai SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE azureopenai;
END

CREATE DATABASE azureopenai;
GO

USE azureopenai;
GO
F:\\Presentation\\t-sql-as-prompts\\tsql\\create_database.sql
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg')
BEGIN
    EXEC('CREATE SCHEMA stg')
END
GO
F:\\Presentation\\t-sql-as-prompts\\tsql\\create_stage_schema.sql
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'prd')
BEGIN
    EXEC('CREATE SCHEMA prd')
END
GO
F:\\Presentation\\t-sql-as-prompts\\tsql\\creates_prd_chema.sql
Sending request for summary to Azure OpenAI endpoint...


IF OBJECT_ID('stg.salestmp', 'U') IS NOT NULL
    DROP TABLE stg.salestmp;

CREATE TABLE stg.salestmp
(
    ProductId VARCHAR(255),

### Create Login and Error Handling process

In [8]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createprocessschema", "create_etl_process_schema.sql"),
    2: ("createprocesslogtable", "create_etl_processlog_table.sql"),
    3: ("createbatcherrorlogtable", "create_etl_errorlog_table.sql"),
    4: ("createprocesslogsp", "create_etl_processlog_usp.sql"),
    5: ("createerrorlogsp", "create_etl_errorlog_usp.sql")
}

### Reading the system and user role prompts

In [9]:
import json
from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

deploymenttype = "CreateLogErrorHandlingObjects" 
index = 1
while index< len(data[deploymenttype]):
    pprint(data[deploymenttype][index])
    index += 1

{'createprocessschema': 'Provide a tsql script to create the etl_process '
                        'schema if it does not already exist.'}
{'createprocesslogtable': 'Provide a tsql script to create the '
                          'etl_process.etl_process_log table if it does not '
                          'already exist with the following fields name id '
                          'integer auto generated identifier without primary '
                          'key, processname varchar 50 lenght, processtype '
                          'varchar 30 lenght, objectname varchar 50 lenght, '
                          'starttime and endtime: DATETIME'}
{'createbatcherrorlogtable': 'Provide a tsql script to create the '
                             'etl_process.error_log table if it does not '
                             'already exist with the following fields name id '
                             'integer auto generated identifier without '
                             'primary key, proces

### Sending request to generate Login and Error Handling Objects
- Tables T-SQL script files
- Store Procedures T-SQL script files

In [10]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)

filedir = ".\\tsql\\"
deploymenttype = "CreateLogErrorHandlingObjects" #"BicylOnPremPrompts" #

index = 1

while index< len(data[deploymenttype]):
    #print(index)
    messages,filename = GeneratePromptFilename(deploymenttype,filedir,data,index,keys)
    if __name__ == '__main__': 
        SendRequestToAzureOpenAI (
         client
        ,azure_openai_api_model
        ,azure_openai_api_temperature
        ,azure_openai_api_max_tokens
        ,messages
        ,filename
        )
    print(filename)
    index+=1

Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT schema_name FROM information_schema.schemata WHERE schema_name = 'etl_process')
BEGIN
    EXEC('CREATE SCHEMA etl_process')
END;
.\tsql\create_etl_process_schema.sql
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'etl_process_log' AND schema_id = SCHEMA_ID('etl_process'))
BEGIN
    CREATE TABLE etl_process.etl_process_log (
        id INT IDENTITY(1,1),
        processname VARCHAR(50),
        processtype VARCHAR(30),
        objectname VARCHAR(50),
        starttime DATETIME,
        endtime DATETIME
    );
END;
.\tsql\create_etl_processlog_table.sql
Sending request for summary to Azure OpenAI endpoint...


IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'error_log' AND schema_id = SCHEMA_ID('etl_process'))
BEGIN
    CREATE TABLE etl_process.error_log
    (
        id INT IDENTITY(1,1),
        processid INT,
        processname VARCHAR(5

### Convert static T-SQL scripts into dynamic T-SQL script files. 

The required steps are:
- Obtain the location of each T-SQL script file.
- Create a Python function to read each T-SQL script file.
- Use Azure OpenAI to make the script re-runnable.

In [11]:
# Create a Python function to read the T-SQL file
def read_tsql_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content

# Define the T-SQL development scripts file names.
keys = {

    1: ("load_staging_data_table_reusable", ".\\tsql\\load_staging_data_table.sql"),
    2: ("load_prd_data_table_reusable", ".\\tsql\\load_prd_data_table.sql"),
}

for each_tsql_file in keys:
    filename= (keys[each_tsql_file][1])
    print("File Name: " + filename)    
    bulk_sql_script = read_tsql_file(filename)
    print(bulk_sql_script)

File Name: .\tsql\load_staging_data_table.sql
TRUNCATE TABLE stg.salestmp;

BULK INSERT stg.salestmp
FROM 'F:\Presentation\t-sql-as-prompts\csv\bicycle_data.csv'
WITH (
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n',
    ERRORFILE = 'F:\Presentation\t-sql-as-prompts\csv\bicycle_data_error.log'
);
File Name: .\tsql\load_prd_data_table.sql
INSERT INTO prd.sales (ProductId, ProductName, ProductType, Color, OrderQuantity, Size, Category, Country, Date, PurchasePrice, SellingPrice)
SELECT 
    CAST(ProductId AS INT),
    ProductName,
    ProductType,
    Color,
    CAST(OrderQuantity AS INT),
    Size,
    Category,
    Country,
    CAST(Date AS DATE),
    CAST(PurchasePrice AS DECIMAL(18,2)),
    CAST(SellingPrice AS DECIMAL(18,2))
FROM stg.salestmp;


In [12]:
import json
# from pprint import pprint

json_file = 'bicycle_data_prompt.json'

with open(json_file) as json_data:
    data = json.load(json_data)


# Define the T-SQL development scripts file names.
keys = {

    1: ("load_staging_data_table_reusable", "load_staging_data_table.sql"),
    2: ("load_prd_data_table_reusable", "load_prd_data_table.sql"),
}

deploymenttype = "MakeDatabaseObjectReusable"

system_role = data[deploymenttype][0]['system_role']
filedir = r"F:\\Presentation\\t-sql-as-prompts\\tsql\\"
index = 1
while index <= len(keys):
    #print(keys[index])
    get_user_input_section = (keys[index][0])
    get_user_input = (data[deploymenttype][index][get_user_input_section])
    tsql_file_path = filedir+(keys[index][1])
    # Read the T-SQL file
    get_tsql_script_from_file = read_tsql_file(tsql_file_path)
    user_prompt = get_user_input + '\n'+ get_tsql_script_from_file
    # Define the messages to be sent to the Azure OpenAI API
    messages = [{"role": "system", "content": system_role},{"role": "user", "content": user_prompt}]
    if __name__ == '__main__': 
        print(tsql_file_path)
        SendRequestToAzureOpenAI(client,azure_openai_api_model,azure_openai_api_temperature,azure_openai_api_max_tokens,messages,tsql_file_path)
    #print(user_prompt)
    index+=1

F:\\Presentation\\t-sql-as-prompts\\tsql\\load_staging_data_table.sql
Sending request for summary to Azure OpenAI endpoint...


CREATE PROCEDURE etl_process.usp_BulkInsertFromCSV
    @tableName NVARCHAR(255),
    @filePath NVARCHAR(255),
    @errorFilePath NVARCHAR(255)
AS
BEGIN
    DECLARE @startTime DATETIME;
    DECLARE @endTime DATETIME;
    DECLARE @errorMsg NVARCHAR(MAX);

    BEGIN TRY
        SET @startTime = GETDATE();

        DECLARE @bulkInsertQuery NVARCHAR(MAX);
        SET @bulkInsertQuery = 'TRUNCATE TABLE ' + @tableName + '; ' +
            'BULK INSERT ' + @tableName + ' ' +
            'FROM ''' + @filePath + ''' ' +
            'WITH (FIRSTROW = 2, FIELDTERMINATOR = '','', ROWTERMINATOR = ''\n'', ERRORFILE = ''' + @errorFilePath + ''');';

        EXEC sp_executesql @bulkInsertQuery;

        SET @endTime = GETDATE();

        EXEC etl_process.usp_get_process_log 'Bulk Insert', 'T-SQL', @tableName, @startTime, @endTime;
    END TRY
    BEGIN CATCH
        SET @endTi

## Loading Data Into SQL Server

### Creating a Python function to access SQL Server

In [13]:
import pyodbc

def execute_sql_script(sql_script):
    # Create a connection string
    conn_str = (
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r"SERVER={az-vm-esi-labs};"
        r'DATABASE=master;'
        r'Trusted_Connection=yes;'
    )
    # Establish a connection with the database
    conn = pyodbc.connect(conn_str)

    # Create a cursor from the connection
    cursor = conn.cursor()

    # Execute the SQL script
    cursor.execute(sql_script)
    # Fetch the results
    results = cursor.fetchall()
    # Commit the transaction if no errors
    conn.commit()

    # Close the connection
    conn.close()
    # Return the results
    return results

### Verifying if SQL Server is accessible

In [15]:
sql_script = "SELECT name FROM sys.databases"
sql_server = "az-vm-esi-labs"
execute_sql_script(sql_script)

[('master',),
 ('tempdb',),
 ('model',),
 ('msdb',),
 ('az_synapse_link',),
 ('dp300_demo_db',),
 ('AdventureWorksLT2019',),
 ('AdventureWorks2019',),
 ('AdventureWorksDW2022',),
 ('AdventureWorks2022',),
 ('AdventureWorksLT2022',),
 ('AdventureWorksDW2020',),
 ('TailspinToys2020-US',),
 ('azureopenai',),
 ('SkillUpAI',)]

### To save time, let’s create some objects and load the data to finish the manual process.

In [16]:
def execute_sql_script_file(server, database, script_file):
    # Establish a connection to the database
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes;')

    # Create a cursor from the connection
    cursor = conn.cursor()

    # Read and execute the script file line by line
    if script_file.endswith(".sql"):
        with open(script_file, "r") as sql_file:
            sql_query = ""
            for line in sql_file:
                # If the line is a GO statement, execute the query and reset it
                if line.strip() == "GO":
                    cursor.execute(sql_query)
                    sql_query = ""
                # Otherwise, append the line to the query
                else:
                    sql_query += line
            # Execute any remaining query
            if sql_query:
                cursor.execute(sql_query)
    else:
        sql_query = script_file
        cursor.execute(sql_query)
    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()


In [17]:
# Initialize T-SQL development scripts file names.
keys = {
    1: ("createprocessschema", ".\\tsql\\create_etl_process_schema.sql"),
    2: ("createstageschema", ".\\tsql\\create_stage_schema.sql"),
    3: ("createsprdchema", ".\\tsql\\creates_prd_chema.sql"),
    4: ("createprocessschema", ".\\tsql\\create_etl_process_schema.sql"),
    5: ("createprocesslogtable", ".\\tsql\\create_etl_processlog_table.sql"),
    6: ("createbatcherrorlogtable", ".\\tsql\\create_etl_errorlog_table.sql"),
    7: ("createprocesslogsp", ".\\tsql\\create_etl_processlog_usp.sql"),
    8: ("createerrorlogsp", ".\\tsql\\create_etl_errorlog_usp.sql"),
    9: ("createstagetable", ".\\tsql\\create_stage_table.sql"),
    10: ("createprdtable", ".\\tsql\\create_prd_table.sql"),
    11: ("loadstagingdatatable", ".\\tsql\\load_staging_data_table.sql"),
    12: ("loadprddatatable", ".\\tsql\\load_prd_data_table.sql"),
    13: ("createview", ".\\tsql\\create_view.sql"),
    14: ("createprocedure", ".\\tsql\\create_procedure.sql")
}

In [18]:
database='SkillUpAI'
server='az-vm-esi-labs'
index = 1
while index<= len(keys):
    print(keys[index][1])
    script_file = keys[index][1]
    execute_sql_script_file(sql_server, database, script_file)
    index+=1


.\tsql\create_etl_process_schema.sql
.\tsql\create_stage_schema.sql
.\tsql\creates_prd_chema.sql
.\tsql\create_etl_process_schema.sql
.\tsql\create_etl_processlog_table.sql
.\tsql\create_etl_errorlog_table.sql
.\tsql\create_etl_processlog_usp.sql
.\tsql\create_etl_errorlog_usp.sql
.\tsql\create_stage_table.sql
.\tsql\create_prd_table.sql
.\tsql\load_staging_data_table.sql
.\tsql\load_prd_data_table.sql
.\tsql\create_view.sql
.\tsql\create_procedure.sql


## **Let us perform the initial CSV bulk load.**
- It will import the CSV file data into SQL Server stage table

In [19]:
# store procedure to load data into stage table
script_file="""
EXECUTE etl_process.usp_BulkInsertFromCSV 
			 'stg.salestmp'
			,'F:\\Presentation\\t-sql-as-prompts\\csv\\bicycle_data.csv'
			,'F:\\Presentation\\t-sql-as-prompts\\csv\\errorload.csv'
"""
database='skillupai'
server='az-vm-esi-labs'
execute_sql_script_file(server, database, script_file)

## **Load Bicycle Sales data from stage to destination**

In [20]:
# store procedure to load data into stage table
script_file="""
EXECUTE etl_process.usp_InsertSalesData
"""
database='skillupai'
server='az-vm-esi-labs'
execute_sql_script_file(server, database, script_file)

iiii

In [21]:
# store procedure to load data into stage table
script_file="""
EXECUTE etl_process.usp_BulkInsertFromCSV 
			 'stg.salestmp'
			,'F:\\Presentation\\t-sql-as-prompts\\csv\\bicycle_data_bad.csv'
			,'F:\\Presentation\\t-sql-as-prompts\\csv\\errorload.csv'
"""
database='skillupai'
server='az-vm-esi-labs'
execute_sql_script_file(server, database, script_file)

In [ ]:
#T-SQL Query to check for the bad data
SELECT *  FROM [skillupai].[stg].[salestmp]

## **Demo TSQL Script**

In [ ]:
%%sql_script
CREATE DATABASE skillupai
GO

-- Copy and past to ssms
SELECT TOP (5) * FROM [skillupai].[stg].[salestmp];
GO

SELECT TOP (5) * FROM [skillupai].[prd].[sales];
GO

SELECT TOP (5) * FROM [skillupai].[etl_process].[etl_process_log];
GO

SELECT TOP (5) * FROM [skillupai].[etl_process].[error_log];
GO

-- View
SELECT TOP (5) * FROM [skillupai].[prd].[vw_GetSaleDetails];
GO

-- Proc
EXECUTE [prd].[usp_GetTotalSalesByCountries]
GO

-- Load bad csv file
--EXECUTE etl_process.usp_InsertSalesData

## **Things to Consider**
When developing a database using prompts as T-SQL development, there are some things to consider, such as:
- **Prompt quality:** The quality of the prompt affects the quality of the generated code. A good prompt should be clear, concise, and specific, and avoid ambiguity, vagueness, and redundancy. A good prompt should also provide enough context and information for the generative AI model to understand the intent and the desired output of the prompt.
Code quality: The quality of the generated code depends on the quality of the generative AI model and the prompt. A good code should be correct, complete, and consistent, and follow the best practices and standards of T-SQL coding. A good code should also be readable, maintainable, and well-documented, and avoid errors, bugs, and vulnerabilities.

- **Code validation:** The generated code should be validated before executing or deploying it to the database. We can use various tools and methods to validate the code, such as syntax checking, code formatting, code analysis, code review, or code testing. We can also compare the generated code with the expected code or the existing code to ensure its accuracy and compatibility.

- **Code modification:** The generated code may not always meet our expectations or requirements, and may need some modification or improvement. We can use various tools and techniques to modify the code, such as code editing, code refactoring, code debugging, or code optimization. We can also use feedback loops or interactive sessions to refine the prompts and the generated code.

## **Microsoft Autogen as a Tool**
Microsoft Autogen is a tool that simplifies the orchestration, automation, and optimization of LLM workflows. It offers customizable and conversable agents that leverage the strongest capabilities of the most advanced LLMs, such as GPT-4, while addressing their limitations by integrating with humans and tools and having conversations between multiple agents via automated chat.

Microsoft Autogen can be a useful tool for database development using prompts as T-SQL development, as it can enable us to:
- **Build complex multi-agent conversation systems:** We can use Microsoft Autogen to define a set of agents with specialized capabilities and roles, such as Commander, Writer, Safeguard, and Executor, and define the interaction behavior between agents, such as what to reply when an agent receives messages from another agent. This way, we can build complex systems that can handle various tasks, such as code-based question answering, supply-chain optimization, conversational chess, and more.

- **Simplify the orchestration, automation, and optimization of LLM workflows:** We can use Microsoft Autogen to simplify the orchestration, automation, and optimization of LLM workflows, such as error handling, multi-config inference, context programming, and more. This way, we can reduce the coding effort, improve the code quality, and enhance the LLM inference.

- **Support diverse conversation patterns for complex workflows:** We can use Microsoft Autogen to support diverse conversation patterns for complex workflows, such as conversation autonomy, the number of agents, and agent conversation topology. This way, we can customize and compose the agents and their interactions to suit our needs and preferences.